In [1]:
import os

import hydra
import torch
import torchaudio
from stable_audio_tools.inference.generation import generate_diffusion_cond

In [2]:
! export all_proxy=127.0.0.1:7890

In [3]:
import os
print(os.path.exists("/cephfs/shared/linzhuo/dataset_maestro/0412/0412.truncated.wav"))


False


In [4]:
import torchaudio
import torch
import os

# 配置
input_wav_path = "/cephfs/shared/linzhuo/dataset_maestro/0412/truncated.wav"
output_wav_path = "/cephfs/shared/linzhuo/dataset_maestro/0412/original.wav"
sample_rate = 44100
target_seconds = 47.0
downsample_rate = 4
sample_size = int(target_seconds * sample_rate / downsample_rate)  # e.g. 51817
max_samples = sample_size * downsample_rate  # waveform length

# 加载并预处理 input/output
def load_audio(path):
    waveform, sr = torchaudio.load(path)
    if sr != sample_rate:
        waveform = torchaudio.functional.resample(waveform, sr, sample_rate)
    if waveform.shape[0] == 1:
        waveform = waveform.repeat(2, 1)
    elif waveform.shape[0] > 2:
        waveform = waveform[:2]
    if waveform.shape[1] > max_samples:
        waveform = waveform[:, :max_samples]
    elif waveform.shape[1] < max_samples:
        pad_len = max_samples - waveform.shape[1]
        waveform = torch.nn.functional.pad(waveform, (0, pad_len))
    return waveform

x = load_audio(input_wav_path).unsqueeze(0).cuda()
y = load_audio(output_wav_path).unsqueeze(0).cuda()

prompt = "in: truncated; out: original"
conditioning = [{
    "audio": y,
    "prompt": prompt,
    "seconds_start": 0.0,
    "seconds_total": target_seconds,
}]


In [5]:
import sys, os
sys.path.append(os.path.abspath("/cephfs/shared/linzhuo/stable-audio-controlnet"))

In [6]:
seed = 42
num_samples = 1
exp_cfg = "train_musdb_controlnet_audio_large_humming"
ckpt_path = "/cephfs/shared/linzhuo/stable-audio-controlnet/logs/ckpts/maestro-controlnet-audio_2025-05-26-21-29-04/last.ckpt"
dataset_path = "/cephfs/shared/linzhuo/processed_maestro"

In [7]:
# load config
with hydra.initialize(config_path="..", version_base=None):
    cond_cfg = hydra.compose(config_name="config", overrides=[f'exp={exp_cfg}',
                                                              f'datamodule.val_dataset.path={dataset_path}/test', 
                                                              f'datamodule.train_dataset.path={dataset_path}/train'])

In [8]:
import os
os.environ["all_proxy"] = "http://127.0.0.1:7890"


In [9]:
# init model
model = hydra.utils.instantiate(cond_cfg["model"])
ckpt = torch.load(ckpt_path, map_location="cpu")
model.load_state_dict(ckpt['state_dict'], strict=False)
model = model.cuda()

/root/miniconda3/envs/gsplat/lib/python3.10/site-packages/x_transformers/x_transformers.py:435: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/root/miniconda3/envs/gsplat/lib/python3.10/site-packages/x_transformers/x_transformers.py:461: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/root/miniconda3/envs/gsplat/lib/python3.10/site-packages/stable_audio_tools/models/transformer.py:126: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/root/miniconda3/envs/gsplat/lib/python3.10/site-packages/stable_audio_tools/models/transformer.py:151: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled =

No module named 'flash_attn'
flash_attn not installed, disabling Flash Attention


/root/miniconda3/envs/gsplat/lib/python3.10/site-packages/vector_quantize_pytorch/vector_quantize_pytorch.py:436: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/root/miniconda3/envs/gsplat/lib/python3.10/site-packages/vector_quantize_pytorch/vector_quantize_pytorch.py:619: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/root/miniconda3/envs/gsplat/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Loaded model from /cephfs/shared/linzhuo/models/stable-audio-1.0/model.safetensors
Loaded model_config from /cephfs/shared/linzhuo/models/stable-audio-1.0/model_config.json


In [10]:
output = generate_diffusion_cond(
    model.model,
    batch_size=1,
    steps=100,
    seed=42,
    cfg_scale=7.0,
    conditioning=conditioning,
    sample_size=sample_size,
    sigma_min=0.3,
    sigma_max=500,
    sampler_type="dpmpp-3m-sde",
    device="cuda"
)

42


/root/miniconda3/envs/gsplat/lib/python3.10/site-packages/stable_audio_tools/models/conditioners.py:314: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=torch.float16) and torch.set_grad_enabled(self.enable_grad):
/root/miniconda3/envs/gsplat/lib/python3.10/site-packages/stable_audio_tools/inference/sampling.py:177: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


  0%|          | 0/100 [00:00<?, ?it/s]

RuntimeError: The size of tensor a (253) must match the size of tensor b (1012) at non-singleton dimension 2

In [ ]:
os.makedirs("out", exist_ok=True)
torchaudio.save("out/input.wav", x[0].cpu(), sample_rate)
torchaudio.save("out/target.wav", y[0].cpu(), sample_rate)
torchaudio.save("out/generated.wav", output[0].cpu(), sample_rate)
